## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Get data from wiki

In [2]:
from bs4 import BeautifulSoup
toronto = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', timeout=5).content, 'html.parser')
toronto

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XklxtApAICoAABwU6N8AAABT","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontari

In [3]:
columns=['Postcode', 'Borough', 'Neighbourhood']
post_table = toronto.find('table')
post_row = post_table.findAll('tr')
db_toronto = pd.DataFrame(columns=columns)

for idx, val in enumerate(post_row):
    post_cell = val.findAll('td')
    df = []
    for idx, val in enumerate(post_cell):
        df.append(val.text.rstrip())
    if(int(len(df)) > 0):
        if(df[1] != "Not assigned"):
            if(df[2] == "Not assigned"):
                df[2] = df[1]

            df_dic={columns[0]: df[0], columns[1]: df[1], columns[2]: df[2]}
            db_toronto = db_toronto.append(df_dic, ignore_index=True)

db_toronto = db_toronto.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))       

db_toronto.shape

(103, 3)

In [4]:
db_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Get  coordinates and add to table

In [5]:
!wget -O geo_data.csv https://cocl.us/Geospatial_data
db_geo = pd.read_csv("geo_data.csv")
db_toronto = db_toronto.join(db_geo)

--2020-02-17 00:14:19--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 158.85.108.83, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-17 00:14:20--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-17 00:14:21--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2sm

In [6]:
db_tor = db_toronto.drop(db_toronto.columns[3], axis=1)
db_tor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Get coordinates for Toronto

In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


### Create Map

In [8]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(db_tor['Latitude'], db_tor['Longitude'], db_tor['Borough'], db_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor


### Connect FourSquare API

In [9]:
scarborough_data = db_tor[db_tor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
address = 'Scarborough, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [13]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
scarborough_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [16]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [17]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e49dd3d1e152c001bb3e96e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Scarborough

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [66]:
# type your answer here

scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )



Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, East Birchmount Park, Ionview
Oakridge, Golden Mile, Clairlea
Scarborough Village West, Cliffside, Cliffcrest
Cliffside West, Birch Cliff
Dorset Park, Wexford Heights, Scarborough Town Centre
Maryvale, Wexford
Agincourt
Tam O'Shanter, Clarks Corners, Sullivan
Milliken, L'Amoreaux East, Steeles East, Agincourt North
L'Amoreaux West
Upper Rouge


In [67]:
print(scarborough_venues.shape)
scarborough_venues.head()

(93, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [68]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Cliffside West, Birch Cliff",4,4,4,4,4,4
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Highland Creek, Port Union, Rouge Hill",1,1,1,1,1,1
"Kennedy Park, East Birchmount Park, Ionview",6,6,6,6,6,6
L'Amoreaux West,17,17,17,17,17,17
"Malvern, Rouge",2,2,2,2,2,2


In [69]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


## 3. Analyze Each Neighborhood

In [70]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Highland Creek, Port Union, Rouge Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
scarborough_onehot.shape

(93, 56)

In [72]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.2,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.2,0.2,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.20,0.0,0.000,0.000000,0.000000,0.0
1,Cedarbrae,0.0,0.125,0.0,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.125,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.125000,0.000000,0.0
2,"Cliffside West, Birch Cliff",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.25,0.000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.25,0.0,0.000,0.000000,0.000000,0.0
3,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000,0.200000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.400000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.2
4,"Guildwood, Morningside, West Hill",0.0,0.000,0.0,0.000,0.000000,0.0,0.125000,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.125,0.000000,0.00,0.0,0.0,0.125,0.0,0.125,0.0,0.0,0.000000,0.0,0.0,0.000000,0.125000,0.0,0.0,0.125,0.000000,0.00,0.0,0.125,0.000000,0.000000,0.0
5,"Highland Creek, Port Union, Rouge Hill",0.0,0.000,0.0,0.000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.0
6,"Kennedy Park, East Birchmount Park, Ionview",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.00,0.000,0.166667,0.0,0.166667,0.00,0.166667,0.000000,0.166667,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.166667,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.0
7,L'Amoreaux West,0.0,0.000,0.0,0.000,0.000000,0.0,0.058824,0.058824,0.0,0.000000,0.00,0.000,0.117647,0.0,0.117647,0.00,0.000000,0.058824,0.000000,0.058824,0.000,0.117647,0.000000,0.000000,0.00,0.117647,0.000,0.000000,0.0,0.058824,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.058824,0.058824,0.0,0.0,0.000,0.058824,0.00,0.0,0.000,0.000000,0.058824,0.0
8,"Malvern, Rouge",0.0,0.000,0.0,0.000,0.000000,

In [73]:
scarborough_grouped.shape

(16, 56)

In [74]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2                Bakery  0.12
3                  Bank  0.12
4       Thai Restaurant  0.12


----Cliffside West, Birch Cliff----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Dorset Park, Wexford Heights, Scarborough Town Centre----
                   venue  freq
0      Indian Restaurant   0.4
1     Chinese Restaurant   0.2
2  Vietnamese Restaurant   0.2
3              Pet Store   0.2
4           Noodle House   0.0


----Guildwood, Morningside, West Hill----
                 venue  freq
0         Intersection  0

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,Cedarbrae,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Hakka Restaurant,Caribbean Restaurant,Discount Store,Cosmetics Shop
2,"Cliffside West, Birch Cliff",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bakery,Bank,Gas Station,Athletics & Sports,Fried Chicken Joint,Fast Food Restaurant
4,"Guildwood, Morningside, West Hill",Intersection,Medical Center,Spa,Electronics Store,Rental Car Location,Mexican Restaurant,Breakfast Spot,Pizza Place,Convenience Store,Coffee Shop


## 4. Cluster Neighborhoods

In [77]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 0, 0, 1, 0, 0, 4, 0, 2, 0, 2, 3, 0, 0], dtype=int32)

In [78]:
db_tor2 = db_tor.rename(columns={'Neighbourhood':'Neighbourhood'}) 
result = scarborough_grouped.join(db_tor2.set_index('Neighbourhood'), on='Neighbourhood')
result

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Postcode,Borough,Latitude,Longitude
0,Agincourt,0.0,0.000,0.0,0.000,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.2,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.2,0.2,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.20,0.0,0.000,0.000000,0.000000,0.0,M1S,Scarborough,43.794200,-79.262029
1,Cedarbrae,0.0,0.125,0.0,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.125,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.125000,0.000000,0.0,M1H,Scarborough,43.773136,-79.239476
2,"Cliffside West, Birch Cliff",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.25,0.000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.25,0.0,0.000,0.000000,0.000000,0.0,M1N,Scarborough,43.692657,-79.264848
3,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000,0.200000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.400000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.2,M1P,Scarborough,43.757410,-79.273304
4,"Guildwood, Morningside, West Hill",0.0,0.000,0.0,0.000,0.000000,0.0,0.125000,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.125,0.000000,0.00,0.0,0.0,0.125,0.0,0.125,0.0,0.0,0.000000,0.0,0.0,0.000000,0.125000,0.0,0.0,0.125,0.000000,0.00,0.0,0.125,0.000000,0.000000,0.0,M1E,Scarborough,43.763573,-79.188711
5,"Highland Creek, Port Union, Rouge Hill",0.0,0.000,0.0,0.000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.0,M1C,Scarborough,43.784535,-79.160497
6,"Kennedy Park, East Birchmount Park, Ionview",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.00,0.000,0.166667,0.0,0.166667,0.00,0.166667,0.000000,0.166667,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.166667,0.0,0.000000,0.000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.0,M1K,Scarborough,43.727929,-79.262029
7,L'Amoreaux West,0.0,0.000,0.0,0.000,0.000000,0.0,0.058824,0.058824,0.0,0.000000,0.00,0.000,0.117647,0.0,0.117647,0.00,0.000000,0.

In [79]:
# # add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# scarborough_merged = scarborough_data

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# scarborough_merged.head() # check the last columns!



scarborough_merged = result

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head() # check the last columns!
scarborough_merged.columns.values

array(['Neighbourhood', 'American Restaurant', 'Athletics & Sports',
       'Auto Garage', 'Bakery', 'Bank', 'Bar', 'Breakfast Spot',
       'Bubble Tea Shop', 'Bus Line', 'Bus Station', 'Café',
       'Caribbean Restaurant', 'Chinese Restaurant', 'Clothing Store',
       'Coffee Shop', 'College Stadium', 'Convenience Store',
       'Cosmetics Shop', 'Department Store', 'Discount Store',
       'Electronics Store', 'Fast Food Restaurant', 'Fried Chicken Joint',
       'Gas Station', 'General Entertainment', 'Grocery Store',
       'Hakka Restaurant', 'Hobby Shop', 'Ice Cream Shop',
       'Indian Restaurant', 'Intersection', 'Italian Restaurant',
       'Korean Restaurant', 'Latin American Restaurant', 'Lounge',
       'Medical Center', 'Metro Station', 'Mexican Restaurant',
       'Middle Eastern Restaurant', 'Motel', 'Noodle House', 'Park',
       'Pet Store', 'Pharmacy', 'Pizza Place', 'Playground', 'Print Shop',
       'Rental Car Location', 'Sandwich Place', 'Skating Rink',
      

In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [ ]:
#### Cluster 1

In [81]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

NameError: name 'manhattan_merged' is not defined